# Market Basket Analysis using Apache Spark

In this notebook, we will perform a Market Basket Analysis using the Instacart dataset.
The dataset contains over 3 million grocery orders from more than 200,000 users.
We will use Apache Spark and its MLlib library to perform frequent pattern mining and association rule mining.

<div style="text-align: center;">
    <img src="images/basket-equation.png" alt="Support, Confidence, and Lift metrics" width="882" height="446">
</div>

## Understanding Support, Confidence, and Lift

In Market Basket Analysis, we use three main metrics to evaluate association rules: Support, Confidence, and Lift.

- **Support** 

This measures how frequently the itemset appears in the dataset. It is calculated as the proportion of transactions that contain the itemset.

$\text{Support}(X \Rightarrow Y) = \frac{\text{freq}(X, Y)}{N}$

Where $\text{freq}(X, Y)$ is the number of transactions containing both $X$ and $Y$, and $N$ is the total number of transactions.

- **Confidence**

This measures how often items in $Y$ appear in transactions that contain $X$.
It is calculated as the proportion of transactions containing $X$ that also contain $Y$.

$\text{Confidence}(X \Rightarrow Y) = \frac{\text{freq}(X, Y)}{\text{freq}(X)}$

Where $\text{freq}(X)$ is the number of transactions containing $X$.

- **Lift**

This measures the strength of an association rule over the random co-occurrence of $X$ and $Y$, providing an indicator of the importance of the rule.

$\text{Lift}(X \Rightarrow Y) = \frac{\text{Support}(X \Rightarrow Y)}{\text{Supp}(X) \times \text{Supp}(Y)}$
  
Where $\text{Supp}(X)$ and $\text{Supp}(Y)$ are the individual supports of $X$ and $Y$.

## Importing Libraries

First, we need to import the necessary libraries.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_set, expr
from pyspark.ml.fpm import FPGrowth
from wordcloud import WordCloud

## Initializing Spark Session

We will now initialize a Spark session with the necessary configurations.

In [ ]:
spark = (SparkSession.builder
         .appName("basket-analysis")
         .enableHiveSupport()
         .config("spark.driver.memory", "3g")
         .config("spark.executor.memory", "3g")
         .getOrCreate())

## Loading the Dataset

Next, we will load the Instacart dataset into Spark DataFrames.
This dataset includes several CSV files, each containing different aspects of the order data.

The dataset can be found on [Kaggle](https://www.kaggle.com/c/instacart-market-basket-analysis/data).

Due to GitHub's file size limits, the dataset is not included directly in the repository.
Please download the dataset from Kaggle, extract the files, and place them in the `data` folder according to the paths specified in the Spark read operations.

In [ ]:
INSTACART_DATA = "../data/instacart"

In [ ]:
aisles = spark.read.csv(f"{INSTACART_DATA}/aisles.csv", header=True, inferSchema=True)
departments = spark.read.csv(f"{INSTACART_DATA}/departments.csv", header=True, inferSchema=True)
order_products_prior = spark.read.csv(f"{INSTACART_DATA}/order_products__prior.csv", header=True, inferSchema=True)
order_products_train = spark.read.csv(f"{INSTACART_DATA}/order_products__train.csv", header=True, inferSchema=True)
orders = spark.read.csv(f"{INSTACART_DATA}/orders.csv", header=True, inferSchema=True)
products = spark.read.csv(f"{INSTACART_DATA}/products.csv", header=True, inferSchema=True)

## Creating Temporary Views

We will create temporary views for each of the DataFrames to enable SQL querying.

In [ ]:
aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
order_products_prior.createOrReplaceTempView("order_products_prior")
order_products_train.createOrReplaceTempView("order_products_train")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")

## Exploratory Data Analysis

### Viewing the First Few Rows of Each Table

Let's take a look at the first few rows of each of the imported files to understand their structure and contents.


In [ ]:
orders.show(n=5)

In [ ]:
products.show(n=5)

In [ ]:
aisles.show(n=5)

In [ ]:
departments.show(n=5)

In [ ]:
order_products_train.show(n=5)

In [ ]:
order_products_prior.show(n=5)

### Orders by Hour of the Day

We will now analyze the distribution of orders by the hour of the day.

In [ ]:
query = """
SELECT COUNT(order_id) AS total_orders, order_hour_of_day AS hour 
FROM orders 
GROUP BY order_hour_of_day 
ORDER BY order_hour_of_day
"""
orders_by_hour = spark.sql(query)
orders_by_hour.show(10)

### Visualizing Orders by Hour

Let's visualize the number of orders placed at each hour of the day using a line plot.

In [ ]:
orders_by_hour_plot = orders_by_hour.toPandas()

sns.set_theme(style="whitegrid")

plt.figure(figsize=(10, 6))
sns.lineplot(x='hour', y='total_orders', data=orders_by_hour_plot, marker='o', markersize=6)

for i in range(orders_by_hour_plot.shape[0]):
    plt.scatter(orders_by_hour_plot['hour'][i], orders_by_hour_plot['total_orders'][i], s=200, facecolors='none', edgecolors='r')

plt.title('Total Orders by Hour of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Total Orders')
plt.xticks(range(24))
plt.show()

### Orders by Days Since Prior Order

Next, we will analyze how often customers place orders by looking at the days since their prior order.

In [ ]:
query = """
SELECT days_since_prior_order, COUNT(order_id) AS total_orders
FROM orders 
GROUP BY days_since_prior_order 
ORDER BY days_since_prior_order
"""
days_since_prior_order = spark.sql(query)
days_since_prior_order.show(10)

### Visualizing Orders by Days Since Prior Order

We will visualize the distribution of orders based on the days since the prior order using a bar plot.

In [ ]:
days_since_prior_order_plot = days_since_prior_order.toPandas()


sns.set_theme(style="whitegrid")

plt.figure(figsize=(10, 6))
sns.barplot(x='days_since_prior_order', y='total_orders', data=days_since_prior_order_plot, 
            palette='viridis', hue='days_since_prior_order', dodge=False, legend=False)

plt.title('Total Orders by Days Since Prior Order')
plt.xlabel('Days Since Prior Order')
plt.ylabel('Total Orders')
plt.show()

### Orders by Day of the Week

Now, let's analyze on which day of the week customers make the most purchases.

In [ ]:
query = """
SELECT COUNT(order_id) AS total_orders, 
  (CASE 
     WHEN order_dow = '0' THEN 'Sunday'
     WHEN order_dow = '1' THEN 'Monday'
     WHEN order_dow = '2' THEN 'Tuesday'
     WHEN order_dow = '3' THEN 'Wednesday'
     WHEN order_dow = '4' THEN 'Thursday'
     WHEN order_dow = '5' THEN 'Friday'
     WHEN order_dow = '6' THEN 'Saturday'              
   end) as day_of_week 
  FROM orders  
 GROUP BY order_dow 
 ORDER BY total_orders desc
"""
order_by_weekday = spark.sql(query)
order_by_weekday.show()

### Visualizing Orders by Day of the Week

We will visualize the number of orders placed on each day of the week using a bar plot.

In [ ]:
order_by_weekday_plot = order_by_weekday.toPandas()

sns.set_theme(style="whitegrid")

plt.figure(figsize=(10, 6))
sns.barplot(x='day_of_week', y='total_orders', hue='day_of_week', data=order_by_weekday_plot,
            palette='viridis', dodge=False, legend=False)

plt.title('Total Orders by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Orders')
plt.show()

### Creating a Master Table

Let's create a master table by merging the product, department, order_products_train, and order_products_prior datasets.
This will help us in further analysis.

In [ ]:
spark.sql("DROP TABLE IF EXISTS order_items_temp")

query = """
CREATE TABLE order_items_temp AS
(
    SELECT order_products.*, products.product_name, products.aisle_id, products.department_id, departments.department
    FROM
    (
        SELECT * FROM order_products_train 
        UNION
        SELECT * FROM order_products_prior
    ) AS order_products
    INNER JOIN products
    ON order_products.product_id = products.product_id
    INNER JOIN departments
    ON products.department_id = departments.department_id
)
"""
spark.sql(query)

### Number of Items per Order

Now, let's find out how many items customers typically purchase in a single order.

In [ ]:
query = """
SELECT order_id, COUNT(product_id) AS total_items
FROM order_items_temp 
GROUP BY order_id
"""
items_by_order = spark.sql(query)
items_by_order.show(10)

In [ ]:
query = """
SELECT total_items, COUNT(order_id) AS num_orders
FROM
(
    SELECT order_id, COUNT(product_id) AS total_items
    FROM order_items_temp 
    GROUP BY order_id
) AS items_by_order
GROUP BY total_items
ORDER BY total_items
"""
items_by_order_aggregated = spark.sql(query)
items_by_order_aggregated.show(10)

### Visualizing Number of Items per Order

We will visualize the distribution of the number of items per order using a bar plot.

In [ ]:
items_by_order_plot = items_by_order_aggregated.toPandas()

sns.set(style="whitegrid")

plt.figure(figsize=(18, 6))
bar_plot = sns.barplot(x='total_items', y='num_orders', data=items_by_order_plot, 
            hue='total_items', palette='viridis', dodge=False, legend=False)

bar_plot.xaxis.set_major_locator(ticker.MultipleLocator(4))
bar_plot.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x/1000)}k'))

plt.title('Number of Orders by Total Items')
plt.xlabel('Total Items')
plt.ylabel('Number of Orders')
plt.show()

### Orders by Department

Next, we will analyze which departments have the most orders.

In [ ]:
query = """
SELECT department, COUNT(*) AS orders_count from order_items_temp
GROUP BY department
ORDER BY orders_count desc
LIMIT 10
"""
orders_by_department = spark.sql(query)
orders_by_department.show()

### Visualizing Orders by Department

We will visualize the top 10 departments based on the number of orders using a pie chart.

In [ ]:
orders_by_department_plot = orders_by_department.toPandas()

plt.figure(figsize=(10, 7))
plt.pie(orders_by_department_plot['orders_count'], labels=orders_by_department_plot['department'],
        autopct='%1.1f%%', startangle=140)

plt.legend(orders_by_department_plot['department'], title="Departments", bbox_to_anchor=(1.05, 1), loc='best')

plt.title('Top 10 Departments by Order Count')
plt.show()

### Most Purchased Products

Let's find out which products are the most purchased by customers.

In [ ]:
query = """
SELECT product_name, COUNT(*) AS orders_count from order_items_temp
GROUP BY product_name
ORDER BY orders_count desc
LIMIT 200
"""
product_by_order = spark.sql(query)
product_by_order.show(10)

In [ ]:
query = """
SELECT product_name
FROM (
  SELECT product_name, count(*) AS orders_count
  FROM order_items_temp
  GROUP BY product_name
  ORDER BY orders_count DESC
  LIMIT 200
)
"""
words_df = spark.sql(query)
words = words_df.rdd.flatMap(lambda x: x).collect()
words_str = ' '.join(words)
word_cloud = WordCloud(background_color="white").generate(words_str)

### Visualizing Most Purchased Products

We will create a word cloud to visualize words related to the most frequently purchased products.

In [ ]:
plt.figure(figsize=(14, 10))
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()
display()

## FP-Growth Algorithm

The FP-Growth algorithm is used for frequent pattern mining.
Let's implement this algorithm to find frequent itemsets and association rules in our dataset.

### Organizing Data by Shopping Basket

First, we need to organize our data into shopping baskets.
We will create a DataFrame where each row represents a single order and contains a list of items in that order.

In [ ]:
query = """
SELECT products.product_name, order_products.order_id 
FROM products 
INNER JOIN order_products_train AS order_products  
WHERE order_products.product_id = products.product_id
"""

raw_data = spark.sql(query)
raw_data.show(5, truncate=False)

In [ ]:
baskets = raw_data.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets.createOrReplaceTempView('baskets')
baskets.show(5, truncate=False)

In [ ]:
query = """
SELECT items from baskets
""" 

baskets_items = spark.sql(query).withColumn('items', expr('TRANSFORM(items, x -> CAST(x AS STRING))'))
baskets_items.show(5, truncate=False)

### Implementing FP-Growth Algorithm

Next, we will use the FP-Growth algorithm from PySpark's MLlib to find frequent itemsets and generate association rules.

In [ ]:
fpgrowth = FPGrowth().setItemsCol("items").setMinSupport(0.001).setMinConfidence(0)
model = fpgrowth.fit(baskets_items)

### Frequent Itemsets

Let's take a look at the most frequent itemsets found by the FP-Growth algorithm.

In [ ]:
most_popular_item_in_basket = model.freqItemsets
most_popular_item_in_basket.createOrReplaceTempView("most_popular_item_in_basket")

In [ ]:
if_then = model.associationRules
if_then.createOrReplaceTempView("if_then")

In [ ]:
query = """
SELECT items, freq
FROM most_popular_item_in_basket 
WHERE SIZE(items) > 2 
ORDER BY freq desc
LIMIT 20
"""

items_freq = spark.sql(query)
items_freq.show(5, truncate=False)

### Association Rules

We will now examine the association rules generated by the FP-Growth algorithm, focusing on those with high confidence and lift.

### High Confidence Rules

Here are the association rules with the highest confidence values.

In [ ]:
query = """
SELECT antecedent AS `antecedent (if)`, consequent AS `consequent (then)`, confidence 
FROM if_then 
ORDER BY confidence DESC
LIMIT 20
"""

confidence = spark.sql(query)
confidence.show(5, truncate=False)

### High Lift Rules

Finally, let's look at the association rules with the highest lift values.

In [ ]:
query = """
SELECT * FROM if_then 
WHERE lift > 1
ORDER BY lift DESC
"""

lift = spark.sql(query)
lift.show(5, truncate=False)

## Step 12: Closing the Spark Session

Finally, we close the Spark session to release resources.

In [ ]:
spark.stop()